# Sample satellite data

Some graphical tests of current primary functions using the {func}`sample satellite data <tams.load_example_tb>`.

In [ ]:
import warnings

import cartopy
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import tams

warnings.filterwarnings("ignore", category=cartopy.io.DownloadWarning)

%matplotlib inline

xr.set_options(display_expand_data=False)

## Load data

In [ ]:
tb = tams.load_example_tb().isel(time=slice(4))
tb

In [ ]:
tb.isel(time=0).plot(x="lon", y="lat", size=5, aspect=2.5);

## Identify cloud elements (CEs)

In [ ]:
times = tb.time
contour_sets, contour_sets_219 = tams.identify(tb)
contour_sets[0].head()

In [ ]:
contour_sets_219[0].head()

In [ ]:
# Simple plot to test 219 matching
m, n = 0, 1  # time, contour #
fig, ax = plt.subplots()
c = contour_sets[m].iloc[[n]]
c.plot(ax=ax)
c.cs219.plot(color="red", ax=ax, alpha=0.4);

## Track CE groups between times

In [ ]:
cs = tams.track(contour_sets, times, u_projection=-5).reset_index(drop=True)
cs.head()

In [ ]:
tams.plot_tracked(cs)

In [ ]:
# CTT movie -- WIP

import cartopy.crs as ccrs
import regionmask

i = -1  # select time
size = 3
cx = cy = 3

tb_ = tb.isel(time=i)
if cx > 1 or cy > 1:
    tb_ = tb_.coarsen(x=cx, y=cy, boundary="trim").mean()

x0, x1 = tb_.lon.values.min(), tb_.lon.values.max()
y0, y1 = tb_.lat.values.min(), tb_.lat.values.max()

aspect = (x1 - x0) / (y1 - y0)
proj = ccrs.Mercator()
tran = ccrs.PlateCarree()

fig = plt.figure(figsize=(size * aspect, size + 1))
gs = fig.add_gridspec(
    2, 2,
    width_ratios=(1, 1), height_ratios=(aspect * 2 + 1, 1),
    left=0.1, right=0.9, bottom=0.1, top=0.9,
    wspace=0.05, hspace=0.18,
)

ax = fig.add_subplot(gs[0, :], projection=proj)
ax.set_extent([x0, x1, y0, y1], crs=tran)
ax.gridlines(draw_labels=True)

ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])

# Background -- CTT
tb_.plot(
    x="lon", y="lat",
    cmap="gray_r", ax=ax, cbar_ax=ax2,
    transform=tran,
    cbar_kwargs=dict(orientation="horizontal"),
)

# CEs with colored precip
shapes = cs[["geometry"]]
regions = regionmask.from_geopandas(shapes)
print("masking")
mask = regions.mask(tb_)
masked = tb_.where(mask >= 0)
print("plotting masked")
masked.plot.pcolormesh(
    x="lon", y="lat",
    ax=ax, cbar_ax=ax3, transform=tran, alpha=0.6,
    cbar_kwargs=dict(orientation="horizontal"),
)

# Tracks up to this time
print("plotting tracks")
t = tb_.time.values
assert np.isscalar(t)
for _, g in cs.groupby("mcs_id"):
    g_ = g[g.time <= t]
    c = g_.to_crs("EPSG:32663").centroid.to_crs("EPSG:4326")
    ax.plot(c.x, c.y, ".-", c="r", lw=2, transform=tran)

## Classify

In [ ]:
cs = tams.classify(cs)
cs.head()